In [1]:
library(dplyr)
library(readxl)
require(reshape)
library(RCurl)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: reshape


Attaching package: ‘reshape’


The following object is masked from ‘package:dplyr’:

    rename




In [2]:
x <- getURL('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')
infectados <- read.csv(text = x)

In [3]:
y <- getURL('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv')
muertos <- read.csv(text=y)

In [4]:
muertos = cbind(muertos[,1],data.frame(matrix(0,nrow=dim(muertos)[1],ncol=19)),muertos[-1])

In [5]:
datos = muertos
num = datos[,-1]
nm= num[,1]
i=2

while (i<dim(num)[2]+1){
    nm=cbind(nm,num[,i]-num[,i-1])
    i=i+1
}

casi=cbind(muertos[1],nm)
i=1
dat=c()

while(i<17+1){
    aux=cbind(matrix(rep(casi[i,1]),length(casi[1,][-1])),t(casi[i,][-1]))
    dat=rbind(dat,aux)
    i=i+1
}

datos=data.frame(dat)
b = rename(datos, c(X1='new_deaths',V1='country'))

In [6]:
datos = infectados
num = datos[,-1]
nm= num[,1]
i=2

while (i<dim(num)[2]+1){
    nm=cbind(nm,num[,i]-num[,i-1])
    i=i+1
}

casi=cbind(muertos[1],nm)
i=1
dat=c()
while(i<17+1){
    aux=cbind(t(casi[i,][-1]))
    dat=rbind(dat,aux)
    i=i+1
}
inf =dat
c = data.frame(inf)
d = rename(c, c(X1='new_cases'))
df = cbind(b,d)

In [7]:
write.csv(df,file = "res.csv",row.names = F)

In [8]:
muTransform <- function(zMedian){
  mu <- log(zMedian) 
}
sigmaTransform <- function(zMean, mu){
  sigma <- sqrt(2*(log(zMean) - mu))
}
# Hospitalisation to death distribution
hospitalisation_to_death_truncated <- function(x, mu, sigma) {
  plnorm(x + 1, mu, sigma) - plnorm(x, mu, sigma)
}
hospitalisation_to_death_truncated_low <- function(x){
  hospitalisation_to_death_truncated(x, muLow, sigmaLow)
}
hospitalisation_to_death_truncated_mid <- function(x){
  hospitalisation_to_death_truncated(x, muMid, sigmaMid)
}
hospitalisation_to_death_truncated_high <- function(x){
  hospitalisation_to_death_truncated(x, muHigh, sigmaHigh)
}
# Función del CFR
scale_cfr <- function(data_1_in, delay_fun){
  case_incidence <- data_1_in$new_cases
  death_incidence <- data_1_in$new_deaths
  cumulative_known_t <- 0 # cumulative cases with known outcome at time tt
  # Sum over cases up to time tt
  for(ii in 1:nrow(data_1_in)){
    known_i <- 0 # number of cases with known outcome at time ii
    for(jj in 0:(ii - 1)){
      known_jj <- (case_incidence[ii - jj]*delay_fun(jj))
      known_i <- known_i + known_jj
    }
    cumulative_known_t <- cumulative_known_t + known_i # Tally cumulative known
  }
  # naive CFR value
  b_tt <- sum(death_incidence)/sum(case_incidence) 
  # corrected CFR estimator
  p_tt <- sum(death_incidence)/cumulative_known_t
  data.frame(nCFR = b_tt, cCFR = p_tt, total_deaths = sum(death_incidence), 
             cum_known_t = round(cumulative_known_t), total_cases = sum(case_incidence))
    }
underReportingEstimates <- function(data, delay_fun){ 
  dplyr::group_by(data, country) %>%
    dplyr::do(scale_cfr(., delay_fun)) %>%
    dplyr::filter(cum_known_t > 0 & cum_known_t >= total_deaths)  %>%
    dplyr::mutate(nCFR_UQ = binom.test(total_deaths, total_cases)$conf.int[2],
                  nCFR_LQ = binom.test(total_deaths, total_cases)$conf.int[1],
                  cCFR_UQ = binom.test(total_deaths, cum_known_t)$conf.int[2],
                  cCFR_LQ = binom.test(total_deaths, cum_known_t)$conf.int[1],
                  underreporting_estimate = cCFRBaseline / (100*cCFR),
                  lower = cCFREstimateRange[1] / (100 * cCFR_UQ),
                  upper = cCFREstimateRange[2] / (100 * cCFR_LQ),
                  quantile25 = binom.test(total_deaths, cum_known_t, conf.level = 0.5)$conf.int[1],
                  quantile75 = binom.test(total_deaths, cum_known_t, conf.level = 0.5)$conf.int[2]) %>% 
    dplyr::filter(total_deaths > 0)}

In [9]:
cCFRBaseline <- 1.4
cCFREstimateRange <- c(1.2, 1.7)
zmeanLow <- 8.7
zmedianLow <- 6.7
muLow <- muTransform(zmedianLow)
sigmaLow <- sigmaTransform(zmeanLow, muLow)
zmeanMid <- 13
zmedianMid <- 9.1
muMid <- muTransform(zmedianMid)
sigmaMid <- sigmaTransform(zmeanMid, muMid)
zmeanHigh <- 20.9
zmedianHigh <- 13.7
muHigh <- muTransform(zmedianHigh)
sigmaHigh <- sigmaTransform(zmeanHigh, muHigh)

In [10]:
allTogetherClean=read.csv('res.csv')

allTogetherLow <- underReportingEstimates(allTogetherClean, hospitalisation_to_death_truncated_low) 
allTogetherMid <- underReportingEstimates(allTogetherClean, hospitalisation_to_death_truncated_mid) 
allTogetherHigh <- underReportingEstimates(allTogetherClean, hospitalisation_to_death_truncated_high)

finalRes <- dplyr::tibble(
      country = allTogetherMid$country,
      total_cases = allTogetherMid$total_cases,
      total_deaths = allTogetherMid$total_deaths,
      underreporting_estimate  = pmin(allTogetherLow$underreporting_estimate, allTogetherMid$underreporting_estimate, allTogetherHigh$underreporting_estimate),
      lower = pmin(allTogetherLow$lower, allTogetherMid$lower, allTogetherHigh$lower),
      upper = pmax(allTogetherLow$upper, allTogetherMid$upper, allTogetherHigh$upper))

names(finalRes)<-c('Localidad','Casos_totales','Muertes_Totales','Tasa_de_no_reportados','Cota_inferior_IC','Cota_superior_IC')

In [11]:
i=1
while(i<4+1){
    if (finalRes[i,6]>1){
        finalRes[i,6] <- 1
    }    
    i=i+1
}

In [12]:
allTogetherLow 
allTogetherMid 
allTogetherHigh
finalRes

country,nCFR,cCFR,total_deaths,cum_known_t,total_cases,nCFR_UQ,nCFR_LQ,cCFR_UQ,cCFR_LQ,underreporting_estimate,lower,upper,quantile25,quantile75
<fct>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Antofagasta,0.003436426,0.006195345,1,161,291,0.01899682,8.699899e-05,0.03411913,0.0001572411,2.2597613,0.3517089,108.1142257,0.001785250,0.01663675
Araucanía,0.022893773,0.031947254,25,783,1092,0.03361117,1.486915e-02,0.04677352,0.0207668487,0.4382223,0.2565554,0.8186124,0.027466277,0.03723449
Arica y Parinacota,0.011764706,0.018458814,2,108,170,0.04185009,1.427950e-03,0.06529655,0.0022505995,0.7584453,0.1837769,7.5535430,0.008902393,0.03597821
Biobío,0.006289308,0.007904178,4,506,636,0.01602426,1.716205e-03,0.02011575,0.0021579629,1.7712152,0.5965474,7.8778001,0.005012824,0.01237219
Los Lagos,0.013888889,0.016601405,6,361,432,0.02998383,5.113580e-03,0.03582308,0.0061232322,0.8433021,0.3349796,2.7763115,0.011700291,0.02362359
Los Ríos,0.017964072,0.022347101,3,134,167,0.05159892,3.720056e-03,0.06403256,0.0046409686,0.6264795,0.1874047,3.6630285,0.012903590,0.03783035
Magallanes,0.011804384,0.017295280,7,405,593,0.02416955,4.758785e-03,0.03528591,0.0069765702,0.8094694,0.3400791,2.4367274,0.012564086,0.02383284
Maule,0.031645570,0.056617458,10,177,316,0.05742588,1.527754e-02,0.10144420,0.0274218926,0.2472736,0.1182916,0.6199426,0.043809696,0.07292631
Metropolitana,0.011921216,0.017326545,69,3982,5788,0.01506314,9.286932e-03,0.02187907,0.0135065699,0.8080088,0.5484694,1.2586467,0.015888659,0.01893163


country,nCFR,cCFR,total_deaths,cum_known_t,total_cases,nCFR_UQ,nCFR_LQ,cCFR_UQ,cCFR_LQ,underreporting_estimate,lower,upper,quantile25,quantile75
<fct>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Antofagasta,0.003436426,0.007590372,1,132,291,0.01899682,8.699899e-05,0.04148569,0.0001917832,1.8444419,0.28925637,88.6417659,0.002177036,0.02026837
Araucanía,0.022893773,0.037222771,25,672,1092,0.03361117,1.486915e-02,0.05443016,0.0242174635,0.3761139,0.22046602,0.7019728,0.032011844,0.04336519
Arica y Parinacota,0.011764706,0.022666900,2,88,170,0.04185009,1.427950e-03,0.07969731,0.0027643271,0.6176407,0.15056970,6.1497788,0.010926151,0.04406563
Biobío,0.006289308,0.009111278,4,439,636,0.01602426,1.716205e-03,0.02316396,0.0024880305,1.5365572,0.51804611,6.8327137,0.005778287,0.01425553
Los Lagos,0.013888889,0.019070867,6,315,432,0.02998383,5.113580e-03,0.04099554,0.0070214337,0.7341040,0.29271477,2.4211580,0.013411053,0.02705931
Los Ríos,0.017964072,0.025884239,3,116,167,0.05159892,3.720056e-03,0.07372131,0.0053654683,0.5408697,0.16277520,3.1684093,0.014908291,0.04364694
Magallanes,0.011804384,0.020839073,7,336,593,0.02416955,4.758785e-03,0.04245173,0.0084161312,0.6718149,0.28267398,2.0199305,0.015147785,0.02870742
Maule,0.031645570,0.067803600,10,147,316,0.05742588,1.527754e-02,0.12154532,0.0331014220,0.2064787,0.09872860,0.5135731,0.052790997,0.08765311
Metropolitana,0.011921216,0.020570842,69,3354,5788,0.01506314,9.286932e-03,0.02596441,0.0160409826,0.6805750,0.46217116,1.0597855,0.018865776,0.02247294


country,nCFR,cCFR,total_deaths,cum_known_t,total_cases,nCFR_UQ,nCFR_LQ,cCFR_UQ,cCFR_LQ,underreporting_estimate,lower,upper,quantile25,quantile75
<fct>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Antofagasta,0.003436426,0.01037078,1,96,291,0.01899682,8.699899e-05,0.05667471,0.0002636924,1.3499461,0.21173466,64.4690572,0.002992203,0.02780207
Araucanía,0.022893773,0.04779819,25,523,1092,0.03361117,1.486915e-02,0.06975772,0.0311698258,0.2928981,0.17202397,0.5453993,0.041154513,0.05566886
Arica y Parinacota,0.011764706,0.03122869,2,64,170,0.04185009,1.427950e-03,0.10837052,0.0038071814,0.4483058,0.11073122,4.4652456,0.015024851,0.06034168
Biobío,0.006289308,0.01152193,4,347,636,0.01602426,1.716205e-03,0.02925013,0.0031495084,1.2150741,0.41025465,5.3976678,0.007311319,0.01802268
Los Lagos,0.013888889,0.02404391,6,250,432,0.02998383,5.113580e-03,0.05150326,0.0088573613,0.5822681,0.23299495,1.9193075,0.016903467,0.03405851
Los Ríos,0.017964072,0.03298242,3,91,167,0.05159892,3.720056e-03,0.09333243,0.0068508863,0.4244685,0.12857268,2.4814308,0.019010327,0.05549810
Magallanes,0.011804384,0.02794247,7,251,593,0.02416955,4.758785e-03,0.05661418,0.0112845830,0.5010295,0.21196104,1.5064801,0.020287066,0.03837671
Maule,0.031645570,0.08976088,10,111,316,0.05742588,1.527754e-02,0.15944241,0.0440509500,0.1559699,0.07526228,0.3859168,0.070016562,0.11568405
Metropolitana,0.011921216,0.02691545,69,2564,5788,0.01506314,9.286932e-03,0.03393548,0.0209973911,0.5201474,0.35361216,0.8096244,0.024684036,0.02938834


Localidad,Casos_totales,Muertes_Totales,Tasa_de_no_reportados,Cota_inferior_IC,Cota_superior_IC
<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>
Antofagasta,291,1,1.3499461,0.21173466,1.0000000
Araucanía,1092,25,0.2928981,0.17202397,0.8186124
Arica y Parinacota,170,2,0.4483058,0.11073122,1.0000000
Biobío,636,4,1.2150741,0.41025465,1.0000000
Los Lagos,432,6,0.5822681,0.23299495,2.7763115
Los Ríos,167,3,0.4244685,0.12857268,3.6630285
Magallanes,593,7,0.5010295,0.21196104,2.4367274
Maule,316,10,0.1559699,0.07526228,0.6199426
Metropolitana,5788,69,0.5201474,0.35361216,1.2586467
